<a href="https://colab.research.google.com/github/iamr7d/Deep-Learning/blob/main/Basics_of_Auto_encoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import datasets
import numpy as np
(x_train , y_train),(x_test,y_test) = datasets.fashion_mnist.load_data()

This line loads the Fashion MNIST dataset, which contains grayscale images of 10 different classes of clothing items (e.g., T-shirts, shoes, bags). Each image is
28
×
28
28×28 pixels.

In [5]:
def preprocess(imgs):
  imgs = imgs.astype('float32')/255.0
  imgs = np.pad(imgs,((0,0),(2,2),(2,2)),constant_values =0.0)
  imgs = np.expand_dims(imgs,-1) #
  return imgs

* The line **imgs = imgs.astype('float32') / 255.0** converts pixel values, which range from 0 to 255, to a floating-point range between 0 and 1. This normalization helps the model learn more efficiently by ensuring pixel values are within a consistent range.

* Padding the images: This line pads each
28
×
28
28×28 image to a
32
×
32
32×32 image by adding a 2-pixel border of zeros around each side. Padding is useful here for a few reasons:

* It makes the images compatible with standard models or architectures, which often work better with
32
×
32
32×32 images.
Padding helps the model retain the original image’s features while extending the input size slightly.
* The argument ((0, 0), (2, 2), (2, 2)) means:

> (0, 0): No padding on the first dimension, which represents the number of images in the dataset.

> (2, 2): Add 2 pixels of padding to both the top and bottom in the height dimension.

> (2, 2): Add 2 pixels of padding to both sides in the width dimension.

> Setting constant values: constant_values=0.0 pads with zeros (black pixels), which maintains consistency for the padded border.

> Adding a channel dimension: This step reshapes the data to include a channel dimension, converting each
32
×
32
32×32 image to a shape of
32
×
32
×
1
32×32×1, where 1 represents the grayscale channel.

In [6]:
x_train = preprocess(x_train)
x_test = preprocess(x_test)